In [ ]:
import copy
import time

from cycler import cycler
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch

from dimp.robots import (
    OmniInput, OmniRobot, OmniState, RobotMPCData
)

### Create The Data for the MPC

In [ ]:
ns = 2      # Number of states (x, y)
ni = 2      # Number of inputs (vx, vy)

nc = 50     # Number of control intervals

s0 = cp.Parameter(ns, name="s0")

mpc_data = RobotMPCData(
    nc=nc,
    states_list=[OmniState(s0)] + [OmniState(cp.Variable(ns, name=f"s{k+1}")) for k in range(nc)],
    statesbar_list=[OmniState(s0)] + [OmniState(cp.Parameter(ns)) for _ in range(nc)],
    inputs_list=[OmniInput(cp.Variable(ni, name=f"i{k}")) for k in range(nc)],
    inputsbar_list=[OmniInput(cp.Parameter(ni)) for _ in range(nc)],
)

dt = 0.1
robot = OmniRobot(dt=dt, mpc_data=mpc_data)

# Parameters
p_goal = np.array([10.0, 5.0])
v_max = 10.0

### Create The MPC Problem

$$
\begin{aligned}
&\min_{\substack{s_k, u_k \\ \quad k=1, \, \dots, \, n_c}} \quad &&\sum_{k=1}^{n_c} w_1 (p_k - p_\text{goal})^2 + w_2 u_k^2) \\
&\text{s.t.} && x_{k+1} = A_k x_k + B_k u_k, \\
& && u_k^2 \leq v_\text{max}^2.
\end{aligned}
$$

In [ ]:
weights = cp.Parameter(2, name="weights", nonneg=True)

def create_qcqp(nc: int):
    objective = cp.Minimize(
          0.5 * weights[0] * cp.sum_squares(cp.hstack([mpc_data.statei[i+1] - p_goal for i in range(nc)]))
        + 0.5 * weights[1] * cp.sum_squares(cp.hstack([mpc_data.inputi[i] for i in range(nc)]))
    )

    dynamics_constraints = robot.dt_dynamics_constraint()

    input_constraints = [
        cp.norm(mpc_data.inputi[i], p=2) - v_max <= 0 for i in range(nc)
    ]

    constraints = dynamics_constraints + input_constraints

    problem = cp.Problem(objective, constraints)

    return problem

qcqp_problem = create_qcqp(nc=nc)
print(type(qcqp_problem))
assert qcqp_problem.is_dpp()

### Simulate the Trajectory

In [ ]:
def simulate_to(
    problem: cp.Problem,
    mpc_data: RobotMPCData,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Simulate the robot's motion over the OCP horizon (trajectory optimization).
    The OCP is solved once and the entire state-input trajectory is obtained in one go.

    Args:
        problem (cp.Problem): The optimization problem to solve.
        mpc_data (RobotMPCData): The MPC data containing state and input variables.

    Returns:
        tuple[np.ndarray, np.ndarray]: Arrays of states and inputs over the simulation steps of size [steps, ns] and [steps, ni], respectively.
    """

    states = np.zeros((mpc_data.nc+1, ns))
    inputs = np.zeros((mpc_data.nc, ni))
    
    s0 = np.array([0, 0])
    mpc_data.statei[0].value = s0

    problem.solve()

    mpc_data.statei[0].value = mpc_data.statei[1].value

    mpc_data.update_bar()
    
    states[0, :] = s0
    for i in range(mpc_data.nc):
        states[i+1, :] = mpc_data.statei[i+1].value
        inputs[i, :] = mpc_data.inputi[i].value

    return states, inputs

In [ ]:
def simulate_mpc(
    problem: cp.Problem,
    mpc_data: RobotMPCData,
    n_steps: int = 200,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Simulate the robot's motion over a fixed number of steps (MPC).
    The OCP is solved multiple times and only the first control input is applied at each step.

    Args:
        problem (cp.Problem): The optimization problem to solve.
        mpc_data (RobotMPCData): The MPC data containing state and input variables.
        n_steps (int): The number of steps to simulate.

    Returns:
        tuple[np.ndarray, np.ndarray]: Arrays of states and inputs over the simulation steps of size [steps, ns] and [steps, ni], respectively.
    """

    states = np.zeros((n_steps, ns))
    inputs = np.zeros((n_steps, ni))

    mpc_data.statei[0].value = np.array([0, 0])

    for i in range(n_steps):

        problem.solve()

        mpc_data.statei[0].value = mpc_data.statei[1].value

        mpc_data.update_bar()

        states[i, :] = mpc_data.statei[1].value
        inputs[i, :] = mpc_data.inputi[1].value

    return states, inputs

### Plot the Trajectory

In [ ]:
def plot_trajectory(states: np.ndarray) -> None:
    steps = states.shape[0]

    xm, xM = states[:, 0].min() - 1, states[:, 0].max() + 1
    ym, yM = states[:, 1].min() - 1, states[:, 1].max() + 1

    fig = go.Figure(
        data=[
            go.Scatter(x=states[:, 0], y=states[:, 1],
                mode="lines", name="Trajectory",
                line=dict(width=2, color="rgba(0, 0, 255, 0.5)", dash='dot')),
            go.Scatter(x=[states[0, 0]], y=[states[0, 1]],
                mode="markers", name="Robot",
                marker=dict(color="blue", size=10)),
        ])

    fig.update_layout(width=600, height=450,
        xaxis=dict(
            range=[xm, xM], autorange=False, zeroline=False, scaleanchor="y", title="x [m]"),
        yaxis=dict(
            range=[ym, yM], autorange=False, zeroline=False, title="y [m]"),
        title_text="Trajectory", title_x=0.5,
        updatemenus = [dict(type = "buttons",
            buttons = [
                dict(
                    args = [None, {"frame": {"duration": 10, "redraw": False},
                                    "fromcurrent": True, "transition": {"duration": 10}, "mode": "immediate"}],
                    label = "Play",
                    method = "animate",
                )])],
    )

    fig.update(frames=[
        go.Frame(
            data=[go.Scatter(x=[states[k, 0]], y=[states[k, 1]])],
            traces=[1]
        ) for k in range(steps)])

    fig.show()

    fig.write_html("omni_robot_mpc.html", include_plotlyjs="cdn", full_html=False)

### Plot the Trajectory Optimization Result

In [ ]:
def simulate_and_plot_qcqp():
    weights.value = np.array([1.0, 1.0])
    
    states, inputs = simulate_to(qcqp_problem, mpc_data)

    plot_trajectory(states)

simulate_and_plot_qcqp()

### Create the `CvxpyLayer`

In [ ]:
cvxpylayer = CvxpyLayer(
    qcqp_problem,
    parameters=[weights, s0],
    variables=[mpc_data.statei[i] for i in range(1, nc + 1)] + [mpc_data.inputi[i] for i in range(nc)],
)

w_th = torch.nn.Parameter(torch.tensor([1.0, 1.0]))
s0_th = torch.tensor([0.0, 0.0])

optim = torch.optim.Adam([w_th], lr=1e-2)

solution = cvxpylayer(w_th, s0_th)

solution[0].sum().backward()

print(f"w.grad: {w_th.grad}")

In [ ]:
def task_loss(states, inputs):
    p_goal_th = torch.tensor(p_goal, requires_grad=True)
    
    st_cost = torch.stack([(s - p_goal_th).pow(2).sum() for s in states]).sum()
    in_cost = torch.stack([u.pow(2).sum()               for u in inputs]).sum()
    return 0.5 * 10.0 * st_cost + 0.5 * 1.0 * in_cost

### Matplotlib Options

In [ ]:
default_cycler = (
    cycler(color=['#E41A1C', '#377EB8', '#4DAF4A', '#984EA3', '#FF7F00', '#FFFF33', '#A65628', '#F781BF', '#999999']) +
    # cycler(color=['#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30', '#4DBEEE', '#A2142F']) +
    cycler('linestyle', ['-', '--', '-', '--', '-', '--', '-', '--', '-'])
)

colors = list(default_cycler.by_key()['color'])

textsize = 12
labelsize = 12

plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=textsize)
plt.rc('ytick', labelsize=textsize)
plt.rc('axes', labelsize=labelsize, prop_cycle=default_cycler)
plt.rc('legend', fontsize=textsize)

plt.rc("axes", grid=True, xmargin=0)
plt.rc("grid", linestyle='dotted', linewidth=0.25)

plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
def plot_training_res(history):
    fig, ax = plt.subplots(2, 1)
    ax[0].plot([h['loss'] for h in history])
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Loss")
    ax[0].set_title("Loss Evolution")

    ax11 = ax[1].twinx()
    ax[1].plot([h['w'][0] for h in history],)
    ax[1].set_title("Weights Evolution")
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel(r"$w_0$", color=colors[0])
    ax[1].tick_params(axis='y', labelcolor=colors[0])

    ax11.plot([h['w'][1] for h in history], color=colors[1], linestyle='--')
    ax11.set_ylabel(r"$w_1$", color=colors[1])
    ax11.tick_params(axis='y', labelcolor=colors[1])

    fig.set_constrained_layout(True)
    plt.show()

### Train 1

Compute the loss only over the OCP horizon.

In [ ]:
n_epochs = 200
history  = []

with torch.no_grad():
    w_th.copy_(torch.tensor([1.0, 1.0]))

for epoch in range(n_epochs):
    optim.zero_grad()

    s0_th = torch.zeros_like(s0_th)

    sol = cvxpylayer(w_th, s0_th)
    states = [sol[i] for i in range(nc)]
    inputs = [sol[nc + i] for i in range(nc)]

    loss = task_loss(states, inputs)
    loss.backward()

    optim.step()

    history.append({
        'loss': loss.item(),
        'w': copy.deepcopy(w_th.detach().cpu().numpy()),
        'dw': copy.deepcopy(w_th.grad.detach().cpu().numpy()),
    })
    print(f"Epoch {epoch:2d} | loss = {loss.item():.4f} | w = {w_th.detach().cpu().numpy()}")

plot_training_res(history)

### Train 2

Compute the loss over a rollout of the policy.

In [ ]:
def rollout(layer, initial_state, W, n_steps=200):
    s_t = initial_state
    states, inputs = [], []

    for _ in range(n_steps):
        sol = layer(W, s_t)
        
        nc = int(len(sol) / 2)
        s1 = sol[0]
        u0 = sol[nc]

        states.append(s1)
        inputs.append(u0)

        s_t = s1

    return states, inputs

In [ ]:
nc_short = 1
n_steps_rollout = 1

qcqp_problem_short_hor = create_qcqp(nc=nc_short)

cvxpylayer_short_hor = CvxpyLayer(
    qcqp_problem_short_hor,
    parameters=[weights, s0],
    variables=[mpc_data.statei[i] for i in range(1, nc_short + 1)] + [mpc_data.inputi[i] for i in range(nc_short)],
)

with torch.no_grad():
    w_th.copy_(torch.tensor([1.0, 1.0]))

optim = torch.optim.Adam([w_th], lr=1e-2)

n_epochs = 200
history  = []
for epoch in range(n_epochs):
    optim.zero_grad()

    s0_th = torch.zeros_like(s0_th)
    
    states, inputs = rollout(cvxpylayer_short_hor, s0_th, w_th, n_steps=n_steps_rollout)

    loss = task_loss(states, inputs)
    loss.backward()
    
    optim.step()

    history.append({
        'loss': loss.item(),
        'w': copy.deepcopy(w_th.detach().cpu().numpy()),
        'dw': copy.deepcopy(w_th.grad.detach().cpu().numpy()),
    })
    print(f"Epoch {epoch:2d} | loss = {loss.item():.4f} | w = {w_th.detach().cpu().numpy()}")

plot_training_res(history)

In [ ]:
results = []

for nc_short in [1, 2, 5, 10, 20, 50]:
    print(f"nc_short: {nc_short}")
    for n_steps_rollout in [1, 2, 5, 10, 20, 50, 100]:
        print(f"n_steps_rollout: {n_steps_rollout}")
                
        mpc_data = RobotMPCData(
            nc=nc_short,
            states_list=[OmniState(s0)] + [OmniState(cp.Variable(ns, name=f"s{k+1}")) for k in range(nc_short)],
            statesbar_list=[OmniState(s0)] + [OmniState(cp.Parameter(ns)) for _ in range(nc_short)],
            inputs_list=[OmniInput(cp.Variable(ni, name=f"i{k}")) for k in range(nc_short)],
            inputsbar_list=[OmniInput(cp.Parameter(ni)) for _ in range(nc_short)],
        )
        robot = OmniRobot(dt=dt, mpc_data=mpc_data)

        qcqp_problem_short_hor = create_qcqp(nc=nc_short)

        cvxpylayer_short_hor = CvxpyLayer(
            qcqp_problem_short_hor,
            parameters=[weights, s0],
            variables=[mpc_data.statei[i] for i in range(1, nc_short + 1)] + [mpc_data.inputi[i] for i in range(nc_short)],
        )

        with torch.no_grad():
            w_th.copy_(torch.tensor([1.0, 1.0]))

        optim = torch.optim.Adam([w_th], lr=1e-2)

        n_epochs = 200
        history  = []
        start_time = time.time()
        for epoch in range(n_epochs):
            optim.zero_grad()

            with torch.no_grad():
                s0_th.zero_()
            
            states, inputs = rollout(cvxpylayer_short_hor, s0_th, w_th, n_steps=n_steps_rollout)

            loss = task_loss(states, inputs)
            loss.backward()
            
            optim.step()
            with torch.no_grad():
                w_th.clamp_(min=1e-6)

            history.append({
                'loss': loss.item(),
                'w': copy.deepcopy(w_th.detach().cpu().numpy()),
                'dw': copy.deepcopy(w_th.grad.detach().cpu().numpy()),
            })
            # print(f"Epoch {epoch:2d} | loss = {loss.item():.4f} | w = {w_th.detach().cpu().numpy()}")
            
        
        ratio = w_th.detach().cpu().numpy()[0] / w_th.detach().cpu().numpy()[1]
        results.append({
            'nc_short': nc_short,
            'n_steps_rollout': n_steps_rollout,
            'ratio': ratio,
            'time': time.time() - start_time
        })
        
df = pd.DataFrame(results)
df.to_csv("data/results.csv", index=False)

In [ ]:
df = pd.read_csv("data/results.csv")

fig, ax = plt.subplots(2, 1)

# One line per nc_short
for nc_short, group in df.groupby('nc_short'):
    ax[0].plot(
        group['n_steps_rollout'],
        group['ratio'],
        marker='o',
        label=f'nc_short = {nc_short}'
    )
    ax[0].set_xlabel("n_steps_rollout")
    ax[0].set_ylabel("Ratio")
    ax[0].set_title("Effect of n_steps_rollout and nc_short on w1 / w2 ratio")
    
    ax[1].plot(
        group['n_steps_rollout'],
        group['time'],
        marker='o',
        label=f'nc_short = {nc_short}'
    )
    ax[1].set_xlabel("n_steps_rollout")
    ax[1].set_ylabel("Time [s]")
    ax[1].set_title("Computation Time")

for a in ax:
    a.legend(title='nc_short', loc='upper left', bbox_to_anchor=(1.05, 1))
    a.set_xscale('log')
    a.set_yscale('log')

plt.tight_layout()
plt.show()